# file
用with可以自动调用close（）<br>
## 文件保存：

In [2]:

# f = open('test.txt', 'w')
# f.write('Test\n')
# print('my', 'name', 'is', 'Mike', sep=' ', end='!', file=f)
# f.close()

with open('text.txt', 'w') as f:
    f.write('Test\n') # 两种写法
    print('my', 'name', 'is', 'Mike', sep=' ', end='!', file=f)# 两种写法



## 文件读取：

In [22]:
s = """\
AAA
BBB
CCC
DDD
"""
with open('test.txt', 'w') as f:
    f.write(s)

with open('test.txt', 'r') as f:
    while True:
        line = f.readline()
        print(line, end='')
        if not line:
            break

with open('test.txt', 'r') as f:
    while True:
        chunk = 2
        line = f.read(chunk)
        print(line)
        if not line:
            break

with open('test.txt', 'r') as f:
        print(f.tell())# 显示当前位置
        print(f.read(1)) 
        print(f.read(3)) # 已经读过了就不会读了
        f.seek(5) # 这个是绝对路径，移动到第五个字符之前
        print(f.tell())
        print(f.read(10))


AAA
BBB
CCC
DDD
AA
A

BB
B

CC
C

DD
D


0
A
AA

5
BB
CCC
DDD


### w+与r+区别：<br>
r+：可读可写，若文件不存在，报错；w+: 可读可写，若文件不存在，创建

In [23]:
s = """\
AAA
BBB
CCC
DDD
"""


with open('test.txt', 'w+') as f:
    f.write(s) # 光标已经移动到末尾了
    f.seek(0) # 不归0读不出来。
    print(f.read())

AAA
BBB
CCC
DDD



## 读取模版

In [25]:
import string

with open('email_template.txt') as f:
    t = string.Template(f.read())
    
contents = t.substitute(name='Mike', contents='How are you?')
print(contents)

Hi Mike.

How are you?

Have a good day.


## 读取csv

In [27]:
import csv

with open('test.csv', 'w') as csv_file:
    fieldnames = ['Name', 'Count']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerow({'Name': 'A', 'Count': 1})
    writer.writerow({'Name': 'B', 'Count': 2})

with open('test.csv', 'r') as csv_file:
    reader = csv.DictReader(csv_file)
    for row in reader:
        print(row['Name'], row['Count'])


A 1
B 2


## file的操作<br>
pathlib中的touch创立文件，glob导出文件列表，shutil复制和移除dir。

In [37]:
import os
import pathlib
import glob
import shutil

print(os.path.exists('test.txt')) # 判断存在
print(os.path.isfile('test.txt')) # 这两个没用吧
print(os.path.isdir('test.txt')) # 都输入后缀了，看的出来吧

# os 还有os.rename, os.symlink, os.mkdir, os.rmdir（删除）等功能

pathlib.Path('empty.txt').touch() # 还可以这样新建文件
os.mkdir('test_dir')
shutil.copy('empty.txt','test_dir/empty.txt')
print(glob.glob('test_dir/*')) # 得到文件夹下文件名
os.remove('empty.txt')
shutil.rmtree('test_dir')

print(os.getcwd())

True
True
False
['test_dir/empty.txt']
/Users/huitang/Desktop/python_learning


## 文件打包和展开
tar类型:termial用tar zxvf tst.tar.gz -C/tmp 来展开

In [56]:
import os
import pathlib
import shutil

os.mkdir('test_dir')
pathlib.Path('test_dir/empty.txt').touch()

import tarfile

with tarfile.open('test.tar.gz', 'w:gz') as tr:
    tr.add('test_dir')

with tarfile.open('test.tar.gz', 'r:gz') as tr:
    tr.extractall(path= 'test_tar')
    with tr.extractfile('test_dir/empty.txt') as f:
        print(f.read()) # 啥也没写
        
shutil.rmtree('test_dir') # 清空
shutil.rmtree('test_tar')
os.remove('test.tar.gz')

b''


zip类型：termial用 unzip test.zip -d zzz开展

In [61]:
import glob
import zipfile
import os
import pathlib
import shutil

os.mkdir('test_dir')
pathlib.Path('test_dir/empty.txt').touch()

with zipfile.ZipFile('test.zip', 'w') as z:
    #z.write('test_dir')
    #z.write('test_dir/test.txt')
    for f in glob.glob('test_dir/**', recursive=True): # 读取list
        z.write(f) # 全都写入

with zipfile.ZipFile('test.zip', 'r') as z:
    z.extractall('zzz')
    #with z.open('test_dir/test.txt') as f:
    #    print(f.read())

shutil.rmtree('zzz') # 清空
shutil.rmtree('test_dir')
os.remove('test.zip')

# 临时文件tempfile
自动删除

In [69]:
import tempfile

with tempfile.TemporaryFile(mode='w+') as t: # 关闭后立即销毁
    t.write('hello')
    t.seek(0)
    print(t.read())
    
with tempfile.NamedTemporaryFile(delete=False) as t: # 多个名字
    print(t.name)
    with open(t.name, 'w+') as f:
        f.write('test\n')
        f.seek(0)
        print(f.read())
        
with tempfile.TemporaryDirectory() as td: # 建立目录
    print(td)
    
temp_dir = tempfile.mkdtemp()# 创建一个临时文件。不自行删除
print(temp_dir)

hello
/var/folders/q8/ntcjkbm14nnf31tt2dn9knmc0000gn/T/tmpx4b51f6d
test

/var/folders/q8/ntcjkbm14nnf31tt2dn9knmc0000gn/T/tmpj98wyl5q
/var/folders/q8/ntcjkbm14nnf31tt2dn9knmc0000gn/T/tmp4ikprpwx


## subprocess子进程管理
用来创建和管理子进程（不是线程），并能够与创建的子进程的stdin，stdout，stderr连接通信，获取子进程执行结束后的返回码，在执行超时或执行错误时得到异常。？？？

In [77]:
import os
import subprocess

# os.system('ls')
subprocess.run(['ls', '-al'])

p1 = subprocess.Popen(['ls', '-al'], stdout=subprocess.PIPE)
p2 = subprocess.Popen(
    ['grep', 'test'], stdin=p1.stdout, stdout=subprocess.PIPE)
p1.stdout.close()
output = p2.communicate()[0]
print(output)

b'drwxr-xr-x   6 huitang  staff     192 May 11 13:35 .pytest_cache\n-rw-r--r--   1 huitang  staff    2572 May 11 13:38 lesson_test.ipynb\n-rw-r--r--   1 huitang  staff      22 May 13 10:35 test.csv\n-rw-r--r--   1 huitang  staff      16 May 13 10:28 test.txt\n'


## date表示

In [85]:
import datetime

now = datetime.datetime.now()
print(now)
print(now.isoformat())# 标准表示
print(now.strftime('%d/%m/%y-%H%S%f'))

today = datetime.date.today()
print(today)
print(today.isoformat())
print(today.strftime('%d/%m/%y'))

t = datetime.time(hour=1, minute=10, second=5, microsecond=100) # 间隔
print(t)
print(t.isoformat())
print(t.strftime('%H_%M_%S_%f'))

print(now)
d = datetime.timedelta(weeks=1)
d = datetime.timedelta(days=365)
d = datetime.timedelta(hours=1)
d = datetime.timedelta(minutes=1)
d = datetime.timedelta(seconds=1)
d = datetime.timedelta(microseconds=1)
print(now-d) # 计算时间间隔

2020-05-13 11:33:30.003738
2020-05-13T11:33:30.003738
13/05/20-1130003738
2020-05-13
2020-05-13
13/05/20
01:10:05.000100
01:10:05.000100
01_10_05_000100
2020-05-13 11:33:30.003738
2020-05-13 11:33:30.003737


In [86]:
import time

print('###')
time.sleep(2) # 用来延时
print('###')
print(time.time())

###
###
1589337258.509085


In [89]:
import os
import shutil

file_name = 'test.txt'
if os.path.exists(file_name):
    shutil.copy(file_name,"{}.{}".format(
    file_name, now.strftime('%Y_%m_%d_%H_%H_%S'))) # 添加了创立时间

with open(file_name, 'w') as f:
    f.write('test')

In [2]:
import csv
csv_columns = ['No','Name','Country']
dict_data = [
{'No': 1, 'Name': 'Alex', 'Country': 'India'},
{'No': 2, 'Name': 'Ben', 'Country': 'USA'},
{'No': 3, 'Name': 'Shri Ram', 'Country': 'India'},
{'No': 4, 'Name': 'Smith', 'Country': 'USA'},
{'No': 5, 'Name': 'Yuva Raj', 'Country': 'India'},
]
csv_file = "Names.csv"
try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for data in dict_data:
            print(data)
            writer.writerow(data)
except IOError:
    print("I/O error")

{'No': 1, 'Name': 'Alex', 'Country': 'India'}
{'No': 2, 'Name': 'Ben', 'Country': 'USA'}
{'No': 3, 'Name': 'Shri Ram', 'Country': 'India'}
{'No': 4, 'Name': 'Smith', 'Country': 'USA'}
{'No': 5, 'Name': 'Yuva Raj', 'Country': 'India'}
